In [ ]:
from scipy.integrate import ode
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
import control

In [ ]:
#def parameters

ts = 1e-4
t_end = 0.005
steps = int(1 / ts)
f0 = 50
V_eff = 230 * np.sqrt(2)

vi1 = 230
vi2 = 230

R = 0.4
L = 2.3e-3
C = 10e-6
LT = 2.3e-3
RLoad = 14

t = np.linspace(0, t_end, steps)

num_episodes = 1  # number of simulation episodes

In [ ]:
#def varaibles

R1 = R
L1 = L
C1 = C
R2 = R
L2 = L
C2 = C
LT1 = LT
LT2 = LT
RT1 = R
RT2 = R

In [ ]:
def env_model_ode(t, x):  # , arg):

    # y = array([i1, v1, iT1, i2, v2, iT2])
    i1a = x[0]
    i1b = x[1]
    i1c = x[2]
    v1a = x[3]
    v1b = x[4]
    v1c = x[5]
    iT1a = x[6]
    iT1b = x[7]
    iT1c = x[8]
    i2a = x[9]
    i2b = x[10]
    i2c = x[11]
    v2a = x[12]
    v2b = x[13]
    v2c = x[14]
    iT2a = x[15]
    iT2b = x[16]
    iT2c = x[17]

    # vi1 = V_eff * np.sin(2 * np.pi * f0 * t)
    # vi2 = V_eff * np.sin(2 * np.pi * f0 * t + 0.5)
    vi1 = 230
    vi2 = 230

    iLoad = iT1 + iT2

    di1a = (vi1 - v1) / L1 - R1 / L1 * i1
    dv1 = (i1 - iT1) / C1
    diT1 = v1 / LT1 - RT1 / LT1 * iT1 - RLoad / LT1 * iLoad

    di2 = (vi2 - v2) / L2 - R2 / L2 * i2
    dv2 = (i2 - iT2) / C2
    diT2 = v2 / LT2 - RT2 / LT2 * iT2 - RLoad / LT2 * iLoad
    
    dx = np.array([di1a, di1b, di1c,
                   dv1a, dv1b, dv1c,
                   diT1a, diT1b, diT1c,
                   di2a, di2b, di2c,
                   dv2a, dv2b, dv2c,
                   diT2a, diT2b, diT2c])
    
    return dx




In [ ]:
A1 = np.array([[-R1/L1, -1/L1, 0],
               [1/C1, 0, -1/C1],
               [0, 1/L1, -R1/L1]])

A2 = np.array([[-R2/L2, -1/L2, 0],
               [1/C2, 0, -1/C2],
               [0, 1/L2, -R2/L2]])

AT1 = np.array([[0, 0, 0],
                [0, 0, 0],
                [0, 0, -RT1/LT1]])

AT2 = np.array([[0, 0, 0],
                [0, 0, 0],
                [0, 0, -RT2/LT2]])

In [ ]:
A = np.block([[A1, AT1],
              [AT2, A2]])

In [ ]:
A

In [ ]:
# x = np.array([i1, v1, iT1, i2, v2, iT2])


In [ ]:
B1 = np.array([[1/L1],
              [0],
              [0]])

B2 = np.array([[1/L2],
              [0],
              [0]])

B = np.block([[B1, zeros],
             [zeros, B2]])

In [ ]:
B

In [ ]:
zeros = np.zeros((3,1))

In [ ]:
C1 = np.array([[1, 1, 1]])
C2 = np.array([[1, 1, 1]])
C = np.block([C1, C2])

In [ ]:
C

In [ ]:
sys = control.ss(A, B, C, 0)

In [ ]:
control.step_info(sys)

In [ ]:
i10 = 0
v10 = 0
iT10 = 0
i20 = 0
v20 = 0
iT20 = 0
t0 = 0

x0 = np.array([i10, v10, iT10, i20, v20, iT20])

In [ ]:
u = np.array([vi1, vi2])[:,None] * np.ones((2,10000))

In [ ]:
u

In [ ]:
T, yout = control.forced_response(sys, T=t, U=u, X0=x0)

In [ ]:
plt.plot(t, yout[0,:], label='v1')
# plt.plot(t,result[:steps,0], label = 'i1')
plt.xlabel(r'$t\,/\,\mathrm{s}$')
plt.ylabel('$v_{\mathrm{1}}\,/\,\mathrm{V}$')
# plt.title('{}'.format())
plt.legend()
plt.grid()
# plt.xlim([0, 0.00005])
# plt.ylim([49.25,50.1])
plt.show()